In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
import re
import os
from sklearn.model_selection import train_test_split
tf.__version__
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
# df = pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/train.csv")

In [3]:
# def clean_data(text):
#     text = text.lower()
#     text = re.sub(r"[-=+*\"#@!$%^&()`<>\[\]]","",text)
#     text = re.sub(r"i'm","i am",text)
#     text = re.sub(r"he's","he is",text)
#     text = re.sub(r"she's","she is",text)
#     text = re.sub(r"it's","it is",text)
#     text = re.sub(r"they're","they are",text)
#     text = re.sub(r"there're","there are",text)
#     text = re.sub(r"there's","there is",text)
#     text = re.sub(r"how're","how are",text)
#     text = re.sub(r"what're","what are",text)
#     text = re.sub(r"where're","where ",text)
#     text = re.sub(r"who're","who are",text)
#     text = re.sub(r"that're","that are",text)
#     text = re.sub(r"when're","when are",text)
#     text = re.sub(r"how's","how is",text)
#     text = re.sub(r"what's","what is",text)
#     text = re.sub(r"where's","where is",text)
#     text = re.sub(r"who's","who is",text)
#     text = re.sub(r"that's","that is",text)
#     text = re.sub(r"when's","when is",text)
#     text = re.sub(r"won't","would not",text)
#     text = re.sub(r"nt't","can not",text)
#     text = re.sub(r"\'bout'","about",text)
#     text = re.sub(r"\'till'","untill",text)
#     text = re.sub(r"\'ll","will",text)
#     text = re.sub(r"\'ve","have",text)
#     text = re.sub(r"\'re","are",text)
#     text = re.sub(r"\'d","would",text)
#     text = re.sub(r"\."," ",text)
#     text = re.sub(r"\,"," ",text)
#     text = re.sub(r"\!"," ",text)
#     text = re.sub(r"\?"," ",text)
#     text = re.sub(r"\;"," ",text)
#     text = re.sub(r"\:"," ",text)
#     return text

    
# df['text'] = df['text'].apply(clean_data)
# df['augmented_text'] = df['augmented_text'].apply(clean_data)




In [4]:
df = pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/Copy of data.csv")
train_x, test_x, train_y, test_y = train_test_split(df['augmented_text'], df['text'], test_size=0.33,random_state=4)
val_x, test_x, val_y, test_y = train_test_split(test_x, test_y, random_state=4, test_size=0.5)

In [5]:
df

,Unnamed: 0,text,augmented_text
0,0,barbaric and shocking use of force against,barbaric and shocking use of ofcre agaistn
1,1,pm says energy policy has almost universal sup...,pm assy energy policy has laomst universal sup...
2,2,coronavirus tasmania peter gutwein economy impact,coronavirus tasmania peter gutwein econijy jjpact
3,3,tamworth police continue hunt for missing man,tawortj olicr continue hunt for miesimg man
4,4,help us secure borders iraq says,help us secure bogcers iraq szs
...,...,...,...
922364,153723,retailers fume over lost easter business,retailers fume over lost easfef bjsijess
922365,153724,malcolm turnbull should put his authority on t...,amclolm turnbull should put his authority on t...
922366,153725,miner considers early bronzewing reopening,kine considers fariy bronzewing reopening
922367,153726,rann government curbs internet debate,ramg government curbs ine5net debate


In [6]:
max_length = 10

In [7]:

g_text_tokenizer = Tokenizer()
g_text_tokenizer.fit_on_texts(train_x)
g_text_word_index = g_text_tokenizer.word_index
train_sequences = g_text_tokenizer.texts_to_sequences(train_x)
train_padded = pad_sequences(train_sequences, maxlen = max_length, padding='post')


text_tokenizer = Tokenizer()
text_tokenizer.fit_on_texts(train_y)
text_word_index = text_tokenizer.word_index
test_sequences = text_tokenizer.texts_to_sequences(train_y)
test_padded = pad_sequences(test_sequences, maxlen = max_length, padding='post')





In [8]:
train_padded.shape

(617987, 10)

In [9]:
test_padded.shape

(617987, 10)

In [10]:
val_train_sequences = g_text_tokenizer.texts_to_sequences(val_x)
val_train_padded = pad_sequences(val_train_sequences, maxlen = max_length, padding='post')

val_test_sequences = text_tokenizer.texts_to_sequences(val_y)
val_test_padded = pad_sequences(val_test_sequences, maxlen = max_length, padding='post')

In [11]:
g_vocab_size = len(g_text_word_index)+1
vocab_size = len(text_word_index)+1
print(g_vocab_size, vocab_size)

862683 84191


In [12]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=g_vocab_size, output_dim=512, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(vocab_size,activation='softmax')
])
# learning_rate = 2
optimizer = tf.keras.optimizers.RMSprop()
optimizer = tf.keras.mixed_precision.LossScaleOptimizer(optimizer)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=['accuracy'])

In [13]:
model.summary(expand_nested=True,
    show_trainable=True)

Model: "sequential"
____________________________________________________________________________
 Layer (type)                Output Shape              Param #   Trainable  
 embedding (Embedding)       (None, 10, 512)           44169369  Y          
                                                       6                    
                                                                            
 bidirectional (Bidirectiona  (None, 10, 512)          1574912   Y          
 l)                                                                         
                                                                            
 dropout (Dropout)           (None, 10, 512)           0         Y          
                                                                            
 dense (Dense)               (None, 10, 512)           262656    Y          
                                                                            
 bidirectional_1 (Bidirectio  (None, 10, 512)          1

In [14]:
temp_test_padded = test_padded.reshape((test_padded.shape[0], test_padded.shape[1],1))
# temp_train_padded = train_padded.reshape((train_padded.shape[0], train_padded.shape[1],1))


In [15]:
epoch = 20
model.fit(train_padded, test_padded, batch_size=512, epochs=epoch, validation_data=(val_train_padded,val_test_padded))

Epoch 1/20


/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:439: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 441693696 elements. This may consume a large amount of memory.
  num_elements)


1208/1208 [==============================] - 994s 817ms/step - loss: 4.0537 - accuracy: 0.4909 - val_loss: 6.1665 - val_accuracy: 0.5501
Epoch 2/20
1208/1208 [==============================] - 994s 823ms/step - loss: 2.5153 - accuracy: 0.6905 - val_loss: 6.7667 - val_accuracy: 0.6035
Epoch 3/20
1208/1208 [==============================] - 997s 825ms/step - loss: 2.0530 - accuracy: 0.7449 - val_loss: 7.7668 - val_accuracy: 0.6251
Epoch 4/20
1208/1208 [==============================] - 996s 824ms/step - loss: 1.7863 - accuracy: 0.7745 - val_loss: 7.9537 - val_accuracy: 0.6355
Epoch 5/20
1208/1208 [==============================] - 995s 824ms/step - loss: 1.5867 - accuracy: 0.7967 - val_loss: 9.0317 - val_accuracy: 0.6434
Epoch 6/20
1208/1208 [==============================] - 994s 823ms/step - loss: 1.4242 - accuracy: 0.8152 - val_loss: 9.3846 - val_accuracy: 0.6492
Epoch 7/20
1208/1208 [==============================] - 995s 823ms/step - loss: 1.2993 - accuracy: 0.8303 - val_loss: 9.466

In [51]:
model.save('/content/drive/MyDrive/Colab_Notebooks/20_epoch_accuracy_92_bidirectional_lstm_rnn_2dropout_1dense')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab_Notebooks/20_epoch_accuracy_92_bidirectional_lstm_rnn_2dropout_1dense/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab_Notebooks/20_epoch_accuracy_92_bidirectional_lstm_rnn_2dropout_1dense/assets


In [41]:

test_sample = test_x[:10]

test_sample = g_text_tokenizer.texts_to_sequences(test_sample)
test_sample = pad_sequences(test_sample, maxlen=max_length, padding='post')


In [42]:
test_x[:10]

553828       asx slip bank wesigvh market bhu6dget ucpdajte
774779                humph4yq nwlcolm not so in the mudcle
60183                      sin slows indias psn for victory
211674            aussie briwxoe wins indycar aeasoh opener
274383                 barnett fum8nb ive5 rio bhp ore fesl
577753       wranadn nun sentenced to 30 eyasr for egnocdie
188249       mcfadden makes cyangea in first wqdriors equax
912396                 tsamanain airbnb regulaotin concerns
229399    sxtholic djurch has discrimination case to ajswet
784557             every orda adeth in australia since 1899
Name: augmented_text, dtype: object

In [43]:
test_y[:10]

553828             asx slip bank weigh market budget update
774779                humphrys malcolm not so in the middle
60183                    rain slows indias push for victory
211674            aussie briscoe wins indycar season opener
274383                 barnett fuming over rio bhp ore deal
577753       rwandan nun sentenced to 30 years for genocide
188249       mcfadden makes changes in first warriors squad
912396                 tasmanian airbnb regulation concerns
229399    catholic church has discrimination case to answer
784557             every road death in australia since 1989
Name: text, dtype: object

In [44]:
pre = model.predict(test_sample)
pre.shape

(10, 10, 84191)

In [45]:
def deTokenize(logits):
    index_to_words = {id: word for word, id in text_tokenizer.word_index.items()}
    index_to_words[0] = ''
    pre_index = np.argmax(logits, 1)
    return ' '.join(index_to_words[prediction] for prediction in pre_index)

In [50]:
# print("Iutput:    ",test_x[2])
# print("Actual:    ",test_y[2])
print("Predicted: ",deTokenize(pre[5]))

Predicted:  nun sentenced to 30 years for genocide   
